## import 

In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import numpy as np
import pandas as pd
from kite_trade import *
import os
# import ta
from tqdm import tqdm
import util_fun as uf
from enctoken import get_kite
kite = get_kite()
import sqlite3
from swifter import set_defaults
set_defaults(
    progress_bar=False,

)
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', str)

In [2]:
timerframe_list = [
"day",
"minute",
 "3minute",
 "5minute",
 "10minute",
 "15minute",
 "30minute",
 "60minute",]

i=1
error_list = []
# instument in nse
inst = pd.DataFrame(kite.instruments("NSE"))
inst_filter = inst.query('(name != "")').copy()
inst_filter.rename(columns = {"tradingsymbol":'Symbol'},inplace = True)
inst_filter = inst_filter[inst_filter.segment == 'NSE']
# inst_filter.query("Symbol == 'HDFCBANK'")

nifty_500 = pd.read_csv('../data/ind_nifty500list.csv')
nifty_500 = nifty_500.merge(inst_filter, on = 'Symbol')
# inst_dict = dict(zip(nifty_500.Symbol, nifty_500.instrument_token))
inst_dict = dict(zip(inst_filter.Symbol, inst_filter.instrument_token))

In [14]:
today_date = "2023-05-01"
start_dt = "2023-05-01"
end_dt = "2023-05-27"
time_frame = "day"

# import sql
# conn = sql.create_connection('../data/Nifty_500.db')
# cursor_obj = conn.cursor()
# sql.delete_table(conn,'fivteen_minute')
df_all = pd.DataFrame()
for symbol, instument in tqdm(inst_dict.items()):
    if i:
        try:
            df_all = df_all.append(
                uf.get_data_parllel(
                    kite, instument, time_frame, start_dt, end_dt
                ).assign(stock=symbol)
            )
            # df_day.to_sql('fivteen_minute',conn, if_exists="append",index = True, index_label = 'Date')
            # directory = f'../data/historical/{time_frame}/{symbol}'
            # if not os.path.exists(directory):
            #     os.makedirs(directory)
            # df_day.to_parquet(f'{directory}/part0.parquet')
        except:
            print(instument)
            error_list.append(instument)
    i += 1
print(error_list)


  1%|▏         | 31/2362 [00:10<12:34,  3.09it/s]

2550785


  1%|▏         | 34/2362 [00:11<12:37,  3.07it/s]

3718145


  2%|▏         | 36/2362 [00:11<12:52,  3.01it/s]

...

2604801


 95%|█████████▌| 2244/2362 [11:49<00:33,  3.47it/s]

2606593


100%|██████████| 2362/2362 [12:25<00:00,  3.17it/s]

[2550785, 3718145, 2553089, 2756865, 2598401, 2556673, 2556417, 2556929, 2109185, 2553345, 2868737, 2557185, 2558977, 3774977, 2565377, 2189313, 2109441, 2552833, 2552577, 3495681, 3932929, 2561025, 2559233, 2561281, 2755585, 2562049, 3371009, 2563073, 2564097, 2564353, 2564865, 2597889, 2106113, 2106881, 2565633, 3645185, 2709249, 2709505, 2884097, 2957569, 2109697, 2957057, 2565889, 3102465, 2884353, 2566145, 2786049, 2885633, 2550529, 3645697, 3646209, 2576897, 3371265, 2572801, 2567169, 2572289, 2572545, 2575105, 2567937, 2568449, 2110721, 2566401, 2574593, 2571521, 2574849, 2567681, 2566657, 2571265, 2570753, 2107649, 2571009, 2572033, 2573825, 2710785, 3382785, 3223809, 2753537, 2606849, 2608385, 2108161, 2575873, 2596097, 2108417, 2578177, 5031169, 4551937, 2693889, 2734081, 2580481, 2579201, 2112513, 2579969, 2580993, 2580225, 2580737, 2579713, 3350785, 2579457, 3572737, 4729601, 2585089, 2586625, 2581761, 2585857, 2593793, 2561793, 2594049, 2590721, 2590209, 2587649, 3827969, 

In [3]:
# df_all.to_parquet('may_day.parquet')
df_all = pd.read_parquet('may_day.parquet')
df_all = df_all[df_all.stock.isin(nifty_500.Symbol.unique())]

In [8]:
df_all = df_all[(df_all['Close'] > 100) &(df_all['Close'] < 5000) ]

## consolidating 

In [9]:
df_all.reset_index(inplace=True)
df_all['Date']  = pd.to_datetime(df_all['Date']).dt.tz_localize(None)
df_all['date_only'] = pd.to_datetime(df_all['Date'].dt.date)

## day level

In [14]:
def is_consolidating(df):
    recent_closes = df[-7:]

    max_close = recent_closes["Close"].max()
    min_close = recent_closes["Close"].min()
    # print("the max close {} and min close {}".format(max_close, min_close))
    if min_close > (max_close * 0.985):
        # print("is consolidating {}".format(set(recent_closes['stock'])))
        return True
    return False


def is_breaking_out(df):
    last_close = df[-1:]["Close"].values[0]

    if is_consolidating(df[:-1]):
        recent_closes = df[-8:-1]
        if last_close > recent_closes["High"].max():
            print("is breaking {}".format(set(recent_closes["stock"])))
            breakout_up.extend(list(set(recent_closes["stock"])))
            return True
        if last_close < recent_closes["Low"].min():
            print("is breaking  below {}".format(set(recent_closes["stock"])))
            breakout_below.extend(list(set(recent_closes["stock"])))
            return True
    return False


In [15]:
breakout_up = []
breakout_below = []
df_breakout = df_all.groupby("stock").apply(is_breaking_out).reset_index()

is breaking {'ABB'}
is breaking  below {'APTUS'}
is breaking {'CRISIL'}
is breaking {'MARICO'}
is breaking  below {'ONGC'}
is breaking {'RELIANCE'}
is breaking {'TIINDIA'}


In [16]:
df_breakout[df_breakout[0] == True].stock.unique()

array(['ABB', 'APTUS', 'CRISIL', 'MARICO', 'ONGC', 'RELIANCE', 'TIINDIA'],
      dtype=object)

In [257]:
breakout_below

[]

In [244]:
breakout_below

['AEGISCHEM', 'APTUS', 'ONGC', 'SIS', 'STARHEALTH']

## minute lavel

In [224]:
def is_consolidating(df):
    recent_closes = df
    # print(recent_closes)

    max_close = recent_closes["Close"].max()
    min_close = recent_closes["Close"].min()
    # print("the max close {} and min close {}".format(max_close, min_close))
    if min_close > (max_close * 0.997):
        # print("is consolidating {}".format(set(recent_closes['stock'])))
        return True
    return False


def is_breaking_out(df):
    df_stock_date = pd.DataFrame()
    k = 0
    for i in range(10, 25):
        if is_consolidating(df[k:i]):
            last_close = df[i : i + 1]["Close"].values[0]
            recent_closes = df[k:i]
            # print(recent_closes)
            if last_close > recent_closes["High"].max():
                print("is breaking {}".format(set(recent_closes["stock"])))
                breakout_up.extend(list(set(recent_closes["stock"])))
                print(
                    "Date",
                    df[i : i + 1]["Date"].values[0],
                    "stock",
                    df[i : i + 1]["stock"].values[0],
                )

                # return True
            if last_close < recent_closes["Low"].min():
                print("is breaking  below {}".format(set(recent_closes["stock"])))
                breakout_below.extend(list(set(recent_closes["stock"])))
                print(
                    "Date",
                    df[i : i + 1]["Date"].values[0],
                    "stock",
                    df[i : i + 1]["stock"].values[0],
                )
                # df_stock_date = df_stock_date.append(
                #     pd.DataFrame(
                #         {
                #             "Date": recent_closes["Date"].unique()[0],
                #             "stock": recent_closes["stock"].unique()[0],
                #         }
                #     ),
                #     index=[0],
                # )
                # return True
            k += 1
        # return False


In [225]:
breakout_up = []
breakout_below = []
breakout_up_dict = {}
breakout_below_dict = {}
df_stock_date= pd.DataFrame()
df_breakout = df_all.groupby('stock').apply(is_breaking_out).reset_index()

is breaking {'AXISBANK'}
Date 2023-05-26T11:05:00.000000000 stock AXISBANK
is breaking  below {'BLUEDART'}
Date 2023-05-26T10:15:00.000000000 stock BLUEDART
is breaking  below {'BLUEDART'}
Date 2023-05-26T10:20:00.000000000 stock BLUEDART
is breaking  below {'BPCL'}
Date 2023-05-26T11:05:00.000000000 stock BPCL
is breaking {'BPCL'}
Date 2023-05-26T11:10:00.000000000 stock BPCL
is breaking  below {'CLEAN'}
Date 2023-05-26T10:05:00.000000000 stock CLEAN
is breaking {'ESCORTS'}
Date 2023-05-26T10:05:00.000000000 stock ESCORTS
is breaking  below {'HDFC'}
Date 2023-05-26T11:00:00.000000000 stock HDFC
is breaking  below {'HDFCBANK'}
Date 2023-05-26T11:00:00.000000000 stock HDFCBANK
is breaking  below {'HINDUNILVR'}
Date 2023-05-26T10:15:00.000000000 stock HINDUNILVR
is breaking  below {'ICICIBANK'}
Date 2023-05-26T10:15:00.000000000 stock ICICIBANK
is breaking {'ICICIBANK'}
Date 2023-05-26T10:35:00.000000000 stock ICICIBANK
is breaking {'ICICIBANK'}
Date 2023-05-26T11:05:00.000000000 stock I

In [226]:
breakout_up

['AXISBANK',
 'BPCL',
 'ESCORTS',
 'ICICIBANK',
 'ICICIBANK',
 'INFY',
 'PETRONET',
 'PETRONET',
 'POLYCAB',
 'SHOPERSTOP',
 'TCS']

In [163]:
df_breakout[df_breakout[0] == True].stock.unique()

array(['ABBOTINDIA', 'ADANITRANS', 'ASIANPAINT', 'ATUL', 'BAJAJHLDNG',
       'BANDHANBNK', 'CENTURYTEX', 'CERA', 'CGPOWER', 'DELTACORP',
       'ENGINERSIN', 'GICRE', 'GRANULES', 'GRAPHITE', 'HAL', 'HCLTECH',
       'IDBI', 'IRCTC', 'JSWSTEEL', 'LEMONTREE', 'MCDOWELL-N', 'QUESS',
       'SHREECEM', 'SUMICHEM', 'TANLA', 'TATACHEM'], dtype=object)

In [227]:
breakout_up

['AXISBANK',
 'BPCL',
 'ESCORTS',
 'ICICIBANK',
 'ICICIBANK',
 'INFY',
 'PETRONET',
 'PETRONET',
 'POLYCAB',
 'SHOPERSTOP',
 'TCS']

In [228]:
breakout_below

['BLUEDART',
 'BLUEDART',
 'BPCL',
 'CLEAN',
 'HDFC',
 'HDFCBANK',
 'HINDUNILVR',
 'ICICIBANK',
 'LATENTVIEW',
 'PETRONET',
 'THYROCARE',
 'THYROCARE',
 'VOLTAS']